<h1>Drunk Driving Deaths Compared to Alcohol Consumption and Weather History</h1>

This is my code for the data sciene group project, we both approached the analysis from different angles, then provider our findings as a whole for the paper. So Jacobs code is a lot different then mine.

For our project we wanted to answer questions about DUI deaths and drinking in general nationally. We wanted to compare national drinking to DUI deaths to see if consumption increases deaths, compare weather to both drinking and deaths, along with other variables.
To accomplish these goals, we used several datasets, one on average monthly temperature by US state, one on alcohol consumption per state by liter of ethanol, and one on impaired driving death rates for each state between 2012 and 2014. We tackled this by both independently creating our own visualizations to try to come up with as many correlations and conclusions as possible. The main questions we had where:

* Does a state consuming more alcohol result in larger DUI deaths?
* Does the temperature affect how much a state drinks?
* Does the type of alcohol a state consumes correlate to DUI deaths?
* Does the temperature of a state affect DUI deaths?



In [ ]:
#import all the libraries and files
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
alcohol_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataForFinal/Alcohol_Consumption_US.csv")
dui_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataForFinal/Impaired_Driving_Death_Rate__by_Age_and_Gender__2012___2014__All_States(1).csv")

Here we imported our libraries and our first two datasets

In [ ]:
# Filter alcohol_df to include only years 2012 and 2014
alcohol_df_filtered = alcohol_df[alcohol_df['Year'].isin([2012, 2014])]
alcohol_df_filtered

,State,State_abbrev,Year,Beer (Per capita consumption),Wine (Per capita consumption),Spirits (Per capita consumption),All beverages (Per capita consumption)
2,Alabama,AL,2014,1.13,0.25,0.62,2.00
4,Alabama,AL,2012,1.16,0.24,0.60,2.00
42,Alaska,AK,2014,1.11,0.52,1.16,2.79
44,Alaska,AK,2012,1.17,0.53,1.13,2.83
82,Arizona,AZ,2014,1.18,0.40,0.79,2.37
...,...,...,...,...,...,...,...
1924,West Virginia,WV,2012,1.24,0.10,0.47,1.80
1962,Wisconsin,WI,2014,1.41,0.38,1.18,2.97
1964,Wisconsin,WI,2012,1.46,0.37,1.18,3.02
2002,Wyoming,WY,2014,1.25,0.31,1.16,2.72


Now I filitered the alcohol consumption data set to 2012-2014 to mathc the DUI dataset

In [ ]:
print(dui_df.columns)

Index(['State ', 'Location', 'All Ages, 2012', 'All Ages, 2014',
       'Ages 0-20, 2012', 'Ages 0-20, 2014', 'Ages 21-34, 2012',
       'Ages 21-34, 2014', 'Ages 35+, 2012', 'Ages 35+, 2014', 'Male, 2012',
       'Male, 2014', 'Female, 2012', 'Female, 2014'],
      dtype='object')


In [ ]:
# Create an empty list to store the data
all_data = []

# Iterate over the years
for year in [2012, 2014]:
  # Select the relevant column for the current year
  year_column = f"All Ages, {year}"

  # Extract the state and the value for the current year
  for index, row in dui_df.iterrows():
      all_data.append([row['State '], row[year_column], year]) #For some ungodly reason, there is a space after "State" which took forever to figure out

# Create a new DataFrame from the collected data
new_dui_df = pd.DataFrame(all_data, columns=['State', 'Death_Rate_Per_100k', 'Year'])

# Display the new DataFrame
new_dui_df

,State,Death_Rate_Per_100k,Year
0,Connecticut,2.4,2012
1,Kentucky,4.0,2012
2,South Dakota,5.7,2012
3,Oklahoma,5.6,2012
4,West Virginia,5.6,2012
...,...,...,...
99,Minnesota,1.9,2014
100,Arkansas,4.6,2014
101,Michigan,2.2,2014
102,Rhode Island,NaN,2014


Here I created a completely new Dataframe that is way simpler compared to the original DUI data, since I don't really need all the info about ages

In [ ]:
#drop NaN rows from both dataframes
alcohol_df_filtered = alcohol_df_filtered.dropna()
new_dui_df = new_dui_df.dropna()

In [ ]:
# Merge the two DataFrames
merged_df = pd.merge(alcohol_df_filtered, new_dui_df, on=['State', 'Year'], how='inner')
print(merged_df.columns)

Index(['State', 'State_abbrev', 'Year', 'Beer (Per capita consumption)',
       'Wine (Per capita consumption)', 'Spirits (Per capita consumption)',
       'All beverages (Per capita consumption)', 'Death_Rate_Per_100k'],
      dtype='object')


Here I merged the datasets on State and Year

In [ ]:
#import plotly with mapbox

!pip install plotly

import plotly.express as px
import plotly.io as pio

# Load the token from the file stored in Google Drive
with open('/content/drive/MyDrive/Colab Notebooks/mapbox_token.txt') as f:
    mapbox_token = f.read().strip()

In [ ]:
# Assuming 'merged_df' is your DataFrame
merged_df['Most Popular Alcohol'] = merged_df.apply(lambda row: 'Beer' if row['Beer (Per capita consumption)'] > row['Spirits (Per capita consumption)'] and
                                                    row['Beer (Per capita consumption)'] > row['Wine (Per capita consumption)'] else ('Spirits' if row['Spirits (Per capita consumption)'] > row['Beer (Per capita consumption)'] and
                                                    row['Spirits (Per capita consumption)'] > row['Wine (Per capita consumption)'] else 'Wine'), axis=1) # this sucks
merged_df

,State,State_abbrev,Year,Beer (Per capita consumption),Wine (Per capita consumption),Spirits (Per capita consumption),All beverages (Per capita consumption),Death_Rate_Per_100k,Most Popular Alcohol
0,Alabama,AL,2014,1.13,0.25,0.62,2.00,5.5,Beer
1,Alabama,AL,2012,1.16,0.24,0.60,2.00,5.5,Beer
2,Alaska,AK,2014,1.11,0.52,1.16,2.79,2.8,Spirits
3,Arizona,AZ,2014,1.18,0.40,0.79,2.37,3.0,Beer
4,Arizona,AZ,2012,1.14,0.45,0.84,2.43,3.6,Beer
...,...,...,...,...,...,...,...,...,...
92,West Virginia,WV,2012,1.24,0.10,0.47,1.80,5.6,Beer
93,Wisconsin,WI,2014,1.41,0.38,1.18,2.97,2.9,Beer
94,Wisconsin,WI,2012,1.46,0.37,1.18,3.02,3.6,Beer
95,Wyoming,WY,2014,1.25,0.31,1.16,2.72,8.2,Beer


Here I have a Lambda function that calculates the most popular alcohol in each state and puts it in it's own new column

In [ ]:
#make a Choropleth map showing the deathRate in States
px.set_mapbox_access_token(mapbox_token)

fig = px.choropleth(merged_df,
                    locations='State_abbrev', #This took awhile to figure out, since it needs to be the abbreviation to work
                    locationmode="USA-states",
                    color='Death_Rate_Per_100k',
                    scope="usa",
                    hover_name="State",
                    color_continuous_scale="Viridis",
                    title="Death Rate per 100k by State",
                    animation_frame="Year")
fig.show()

Here we can see that Montana, Wyoming, and North Dakota have large death rates for north states. Conversely New Mexico, Texas, Louisiana, and other southern bordering states also have large death rates. Meanwhile the middle states have relatively low rates.

In [ ]:
#make a Choropleth map showing the most Popular Alchohol in States
px.set_mapbox_access_token(mapbox_token)

fig = px.choropleth(merged_df,
                    locations='State_abbrev',
                    locationmode="USA-states",
                    color='Most Popular Alcohol',
                    scope="usa",
                    hover_name="State",
                    color_discrete_sequence=px.colors.qualitative.Bold,
                    title="Most popular Alcohol in each state",
                    animation_frame="Year")
fig.show()

Here we can see that beer vastly outperforms all other types of alcoholic beverages, with wine being the least popular.

In [ ]:
#make a Choropleth map showing the most consumption in each state
px.set_mapbox_access_token(mapbox_token)

fig = px.choropleth(merged_df,
                    locations='State_abbrev',
                    locationmode="USA-states",
                    color='All beverages (Per capita consumption)',
                    scope="usa",
                    hover_name="State",
                    color_continuous_scale="Viridis",
                    title="Most Consumption",
                    animation_frame="Year")
fig.show()

Above we can see that northern states have lots of consumption, with Utah having virtually none. New Hampshire has a ton of consumption, for some reason. We suspect that that reason has to do with no sales tax causing neighboring states to buy alcohol from them.

In [ ]:
import plotly.express as px

fig = px.scatter(merged_df, x="All beverages (Per capita consumption)", y="Death_Rate_Per_100k", animation_frame="Year",
                 hover_name="State", title="Beverage Consumption vs. Death Rate")
fig.show()

Here we have a scatter plot that shows no coralation between alcohol consumption and DUI deaths.

In [ ]:
temp_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataForFinal/average_monthly_temperature_by_state_1950-2022.csv")
#drop years that aren't 2012
temp_df = temp_df[temp_df['year'].isin([2012, 2014])]
temp_df

,Unnamed: 0,month,year,state,average_temp,monthly_mean_from_1901_to_2000,centroid_lon,centroid_lat
35712,35712,1,2012,Alabama,49.7,45.8,-86.828372,32.789832
35713,35713,1,2012,Arizona,42.6,41.1,-111.664418,34.293110
35714,35714,1,2012,Arkansas,44.0,40.5,-92.439268,34.899745
35715,35715,1,2012,California,44.6,42.6,-119.610699,37.246071
35716,35716,1,2012,Colorado,26.9,24.5,-105.547825,38.998552
...,...,...,...,...,...,...,...,...
37435,37435,12,2014,Virginia,41.4,41.1,-78.812254,37.515024
37436,37436,12,2014,Washington,35.2,33.6,-120.446866,47.380969
37437,37437,12,2014,West Virginia,37.8,37.5,-80.613712,38.642599
37438,37438,12,2014,Wisconsin,24.7,24.9,-90.011694,44.638030


Here we import a new temperature dataset

In [ ]:
#A dictionary to convert to abbreviation
state_abbr = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia': 'DC'
}
#considate to temp per year
average_temp_by_year = temp_df.groupby(['year', 'state']).agg({'month': 'first', 'average_temp': 'mean'}).reset_index()
average_temp_by_year = average_temp_by_year.drop(columns=['month'])
average_temp_by_year['State_abbrev'] = average_temp_by_year['state'].map(state_abbr)
average_temp_by_year

,year,state,average_temp,State_abbrev
0,2012,Alabama,64.850000,AL
1,2012,Arizona,62.000000,AZ
2,2012,Arkansas,63.475000,AR
3,2012,California,59.483333,CA
4,2012,Colorado,48.233333,CO
...,...,...,...,...
91,2014,Virginia,54.400000,VA
92,2014,Washington,48.125000,WA
93,2014,West Virginia,50.700000,WV
94,2014,Wisconsin,39.716667,WI


Here I created a dictionary since plotly Choropleths don't like whole state names, and instead want postal codes.

In [ ]:
#make a Choropleth map showing the temperature in each state
px.set_mapbox_access_token(mapbox_token)

fig = px.choropleth(average_temp_by_year,
                    locations='State_abbrev',
                    locationmode="USA-states",
                    color='average_temp',
                    scope="usa",
                    hover_name="state",
                    color_continuous_scale="Viridis",
                    title="Temperature Average",
                    animation_frame="year")
fig.show()

Here I plotted the temperature data on the Chorpleth and we can see that as we travel North, the states get colder, no surprise there.

In [ ]:
average_temp_by_year = average_temp_by_year.rename(columns={'state': 'State'})
average_temp_by_year = average_temp_by_year.rename(columns={'year': 'Year'})
final_merged_df = pd.merge(merged_df, average_temp_by_year, on=['State', 'Year'], how='inner')
final_merged_df["temp_norm"] = (final_merged_df["average_temp"] - final_merged_df["average_temp"].min()) / (final_merged_df["average_temp"].max() - final_merged_df["average_temp"].min())
final_merged_df["alc_norm"] = (final_merged_df["All beverages (Per capita consumption)"] - final_merged_df["All beverages (Per capita consumption)"].min()) / (final_merged_df["All beverages (Per capita consumption)"].max() - final_merged_df["All beverages (Per capita consumption)"].min())
final_merged_df["deathRate_norm"] = (final_merged_df["Death_Rate_Per_100k"] - final_merged_df["Death_Rate_Per_100k"].min()) / (final_merged_df["Death_Rate_Per_100k"].max() - final_merged_df["Death_Rate_Per_100k"].min())
#final_merged_df["temp_norm"] = 1 - final_merged_df["temp_norm"]
final_merged_df["alc_norm"] = 1 - final_merged_df["alc_norm"]
final_merged_df["deathRate_norm"] = 1 - final_merged_df["deathRate_norm"]
final_merged_df

,State,State_abbrev_x,Year,Beer (Per capita consumption),Wine (Per capita consumption),Spirits (Per capita consumption),All beverages (Per capita consumption),Death_Rate_Per_100k,Most Popular Alcohol,average_temp,State_abbrev_y,temp_norm,alc_norm,deathRate_norm
0,Alabama,AL,2014,1.13,0.25,0.62,2.00,5.5,Beer,61.950000,AL,0.706215,0.807927,0.574257
1,Alabama,AL,2012,1.16,0.24,0.60,2.00,5.5,Beer,64.850000,AL,0.791697,0.807927,0.574257
2,Arizona,AZ,2014,1.18,0.40,0.79,2.37,3.0,Beer,62.141667,AZ,0.711864,0.695122,0.821782
3,Arizona,AZ,2012,1.14,0.45,0.84,2.43,3.6,Beer,62.000000,AZ,0.707689,0.676829,0.762376
4,Arkansas,AR,2014,0.96,0.21,0.62,1.79,4.6,Beer,58.491667,AR,0.604274,0.871951,0.663366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,West Virginia,WV,2012,1.24,0.10,0.47,1.80,5.6,Beer,54.300000,WV,0.480717,0.868902,0.564356
90,Wisconsin,WI,2014,1.41,0.38,1.18,2.97,2.9,Beer,39.716667,WI,0.050847,0.512195,0.831683
91,Wisconsin,WI,2012,1.46,0.37,1.18,3.02,3.6,Beer,47.416667,WI,0.277819,0.496951,0.762376
92,Wyoming,WY,2014,1.25,0.31,1.16,2.72,8.2,Beer,41.850000,WY,0.113731,0.588415,0.306931


Here we merged the temperatures with the rest of the data, and created a normalized value column for the deathrates, alcohol consumption, and the temperatures, so that we can calculate a color for a new visualization

In [ ]:
from pickle import REDUCE
import colorsys

# Create a custom color function for RGB with brightness
def generate_color(row):
    # Calculate the Red and Blue components based on temp_norm (0 to 1)
    red_base = int(row["temp_norm"] * 255)  # Red increases as temp_norm increases
    blue_base = 255 - red_base  # Blue decreases as temp_norm increases

    # Alcohol consumption determines brightness/lightness (0 to 1)
    brightness = row["alc_norm"]
    brightness = max(0.1, min(brightness, 0.9))  # Clamp between 0.1 and 0.9
    # Convert RGB to HLS to apply brightness, then back to RGB
    hls_color = colorsys.rgb_to_hls(red_base / 255.0, 0, blue_base / 255.0)
    rgb_color = colorsys.hls_to_rgb(hls_color[0], brightness, hls_color[2])  # Use brightness for lightness

    # Convert back to 0-255 range for RGB
    red = int(rgb_color[0] * 255)
    green = int(rgb_color[1] * 255)
    blue = int(rgb_color[2] * 255)

    # Debug: Print the RGB values
    #print(f"RGB: ({red}, {green}, {blue}), Brightness: {brightness}")

    # Return the RGB color string
    return f"rgb({red}, {green}, {blue})"

# Apply the function to the dataframe
final_merged_df["color"] = final_merged_df.apply(generate_color, axis=1)

#Do the same but for temperature and DUI deaths
def generate_color(row):
    # Calculate the Red and Blue components based on temp_norm (0 to 1)
    red_base = int(row["temp_norm"] * 255)  # Red increases as temp_norm increases
    blue_base = 255 - red_base  # Blue decreases as temp_norm increases

    # Alcohol consumption determines brightness/lightness (0 to 1)
    brightness = row["deathRate_norm"]
    brightness = max(0.1, min(brightness, 0.9))  # Clamp between 0.1 and 0.9
    # Convert RGB to HLS to apply brightness, then back to RGB
    hls_color = colorsys.rgb_to_hls(red_base / 255.0, 0, blue_base / 255.0)
    rgb_color = colorsys.hls_to_rgb(hls_color[0], brightness, hls_color[2])  # Use brightness for lightness

    # Convert back to 0-255 range for RGB
    red = int(rgb_color[0] * 255)
    green = int(rgb_color[1] * 255)
    blue = int(rgb_color[2] * 255)

    # Debug: Print the RGB values
    #print(f"RGB: ({red}, {green}, {blue}), Brightness: {brightness}")

    # Return the RGB color string
    return f"rgb({red}, {green}, {blue})"

# Apply the function to the dataframe
final_merged_df["color_DUI_Deaths"] = final_merged_df.apply(generate_color, axis=1)

# Filter rows where state is Utah
utah_rows = final_merged_df[final_merged_df["State"] == "Utah"]
print(utah_rows.to_string())

   State State_abbrev_x  Year  Beer (Per capita consumption)  Wine (Per capita consumption)  Spirits (Per capita consumption)  All beverages (Per capita consumption)  Death_Rate_Per_100k Most Popular Alcohol  average_temp State_abbrev_y  temp_norm  alc_norm  deathRate_norm               color    color_DUI_Deaths
81  Utah             UT  2014                           0.66                           0.19                              0.52                                    1.37                  2.0                 Beer     50.066667             UT   0.355932       1.0        0.920792  rgb(231, 204, 254)  rgb(231, 204, 254)
82  Utah             UT  2012                           0.69                           0.19                              0.50                                    1.37                  1.2                 Beer     50.900000             UT   0.380496       1.0        1.000000  rgb(235, 204, 254)  rgb(235, 204, 254)


I now needed a way to represent this 2D data. My solution was to have a color gradient system. The temperature is represented by a gradient between red and blue. Blue means a state is cold, while red means a state is warm, purple would be somewhere in between. Then the color’s saturation represents some other statistic, either alcohol consumption, or death rate. For example in a choropleth that compares alcohol consumption to temperature, a dark blue state would be a heavy drinking, cold state, while a light blue state is a light drinking cold state. Conversely, a dark red state is a heavy drinking warm state and a pink state is a light drinking warm state. This allows for our edge cases to really jump out. The function above calculates these colors and puts them in a new column.

In [ ]:
#make a Choropleth map showing the temperature and alcohol consumption in each state
px.set_mapbox_access_token(mapbox_token)

fig = px.choropleth(final_merged_df,
                    locations='State_abbrev_x',
                    locationmode="USA-states",
                    color='color',
                    scope="usa",
                    hover_name="State",
                    color_discrete_map="identity",
                    custom_data=['Death_Rate_Per_100k', 'average_temp', 'All beverages (Per capita consumption)'],
                    hover_data={'Death_Rate_Per_100k': True, 'average_temp': True, 'All beverages (Per capita consumption)': True, 'State_abbrev_x': False},  # Define hover data and visibility
                    title="Temperature Average: Saturation = Intense Drinking, Color = Temperature",
                    animation_frame="Year")
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
#make a Choropleth map showing the temperature and DUI Deaths in each state
px.set_mapbox_access_token(mapbox_token)

fig = px.choropleth(final_merged_df,
                    locations='State_abbrev_x',
                    locationmode="USA-states",
                    color='color_DUI_Deaths',
                    scope="usa",
                    hover_name="State",
                    color_discrete_map="identity",
                    custom_data=['Death_Rate_Per_100k', 'average_temp', 'All beverages (Per capita consumption)'],
                    hover_data={'Death_Rate_Per_100k': True, 'average_temp': True, 'All beverages (Per capita consumption)': True, 'State_abbrev_x': False},  # Define hover data and visibility
                    title="Temperature Average: Saturation = DUI Deaths, Color = Temperature",
                    animation_frame="Year")
fig.update_layout(showlegend=False)
fig.show()

Here we can see a few interesting things. With the first visualization, we can see that people in the northern states vastly drink more than people in warm, southern states, with two exceptions in Louisiana and Florida. The bottom graph shows us equally useful info. Here we can see that there are pretty equal numbers of saturated red and blue states, telling us that the temperature of the state doesn’t seem to have an effect on the DUI death rates in that state. Here you are just as likely to die of a substance-influenced death in Montana as you are in Texas.
